### Solução utilizando PySpark

In [105]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [111]:
#Import Datasets
df_contracts = spark.read.csv("contracts.csv", header=True, inferSchema=True)
df_transactions = spark.read.csv("transactions.csv", header=True, inferSchema=True)

df_contracts.show()
df_transactions = df_transactions.na.replace("null", "0")
df_transactions.show()

+-----------+---------+---------------+----------+---------+
|contract_id|client_id|    client_name|percentage|is_active|
+-----------+---------+---------------+----------+---------+
|          3|     3545| Magazine Luana|       2.0|     true|
|          4|     3545| Magazine Luana|      1.95|    false|
|          5|     3509|Lojas Italianas|       1.0|     true|
|          6|     3510|     Carrerfive|       3.0|     true|
+-----------+---------+---------------+----------+---------+

+---------------+---------+------------+-------------------+
|transaction_id |client_id|total_amount|discount_percentage|
+---------------+---------+------------+-------------------+
|              1|     3545|        3000|               6.99|
|              2|     3545|        4500|               0.45|
|              3|     3509|       69998|                  0|
|              4|     3510|           1|                  0|
|              5|     4510|          34|                 40|
+---------------+------

In [112]:
#Calcula valor liquido da transacao 
df_liquid_value = df_transactions.selectExpr("client_id", "total_amount - (total_amount * (discount_percentage / 100)) as liquid_value")
df_liquid_value.show()

+---------+------------+
|client_id|liquid_value|
+---------+------------+
|     3545|      2790.3|
|     3545|     4479.75|
|     3509|     69998.0|
|     3510|         1.0|
|     4510|        20.4|
+---------+------------+



In [113]:
#Total de ganhos
df_liquid_value.join(df_contracts, df_liquid_value["client_id"] == df_contracts["client_id"]).filter("is_active = 'true'").selectExpr("sum(liquid_value * (percentage / 100)) as total").show()

+-----------------+
|            total|
+-----------------+
|845.4110000000001|
+-----------------+

